# Build the database from calendars

In [115]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [116]:
from pathlib import Path
import pandas as pd

# set the folder
folder = Path('/run/user/1000/gvfs/smb-share:server=istore,share=microscopy_group_share,user=jeromeb/Admin/Usage Stats/2024')
booking_types = ['default','service', 'maintenance', 'training']
divisions = ['Unknown','Cell Biology','Neurobiology','PNAS','Structural Studies','University']
groups = pd.read_csv(folder/'groups.csv')
users = pd.read_csv(folder/'users.csv')
calendars = pd.read_csv(folder/'calendars.csv')
with open('cookie.txt','r') as f:
    curl_str = f.read()
# curl_str = "curl 'https://mercury.mrc-lmb.cam.ac.uk/ical' --compressed -H 'User-Agent: Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:131.0) Gecko/20100101 Firefox/131.0' -H 'Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/png,image/svg+xml,*/*;q=0.8' -H 'Accept-Language: en-GB,en;q=0.5' -H 'Accept-Encoding: gzip, deflate, br, zstd' -H 'DNT: 1' -H 'Sec-GPC: 1' -H 'Connection: keep-alive' -H 'Referer: https://mercury.mrc-lmb.cam.ac.uk/integration/linux.html?lang=en' -H 'Cookie: SESSION_CONNECT_WEBMAIL=c3667fe143e2dc76a30611b1072880b836143257f462f6a750e324fdef164259; TOKEN_CONNECT_WEBMAIL=f00b0c6e465aedc5468903660dc514453fcd40ed976cb95f506dfc6f5b3eef14' -H 'Upgrade-Insecure-Requests: 1' -H 'Sec-Fetch-Dest: document' -H 'Sec-Fetch-Mode: navigate' -H 'Sec-Fetch-Site: same-origin' -H 'Sec-Fetch-User: ?1' -H 'Priority: u=0, i'"
# cookie = 'SESSION_CONNECT_WEBMAIL=c3667fe143e2dc76a30611b1072880b836143257f462f6a750e324fdef164259; TOKEN_CONNECT_WEBMAIL=f00b0c6e465aedc5468903660dc514453fcd40ed976cb95f506dfc6f5b3eef14'
db_file = 'bookings.db'

Initialize the database

In [121]:
from usagestats.bookingdb import BookingDB

with BookingDB(db_file, curl_str=curl_str) as bookings:
    bookings.create_calendars_from_df(calendars)
    bookings.create_instruments()
    bookings.create_booking_types(booking_types)
    bookings.create_division(divisions)
    bookings.create_groups(groups)
    bookings.create_events()
    # df = bookings.update_users_group_from_df(users)



HTTPError: HTTP Error 404: Not found

In [117]:
with BookingDB(db_file, cookie) as bookings:
    print(bookings.cookie)
    

SESSION_CONNECT_WEBMAIL=c3667fe143e2dc76a30611b1072880b836143257f462f6a750e324fdef164259; TOKEN_CONNECT_WEBMAIL=f00b0c6e465aedc5468903660dc514453fcd40ed976cb95f506dfc6f5b3eef14


In [105]:
from urllib import request
request.Resques

In [89]:
c.split(' ').index("'Cookie: ")

ValueError: "'Cookie: " is not in list

In [5]:
with BookingDB(db_file) as bookings: 
    #bookings.cursor.execute('SELECT * from events')
    #rows = bookings.cursor.fetchall()
    tmp = pd.read_sql_query("SELECT users.name as 'name', groups.name as 'group' from users LEFT JOIN groups ON users.group_id=groups.id", bookings.connection)
    #tmp = pd.read_sql_query("SELECT * from users", bookings.connection)
    #groups = pd.read_sql_query("SELECT * from groups", bookings.connection)
    
#users.join(groups,left_on='group_id',right_on='id')
#rows[:10]
tmp


,name,group
0,Silvia Benito-Kwiecinski,Unknown
1,Nick Barry,Light Microscopy
2,Ben Ravenhill,Unknown
3,Isabel Beets,Unknown
4,Marios Chatzigeorgiou,Unknown
...,...,...
638,Kalina Haas,Unknown
639,Clinton Lau,Unknown
640,Gengjing Zhao,Unknown
641,Dan Trowell,Unknown


In [ ]:
with BookingDB(db_file) as bookings:
    bookings.cursor.execute("""SELECT rowid FROM divisions WHERE name=='Cell Biology'""")
    r = bookings.cursor.fetchone()
    print(bookings.get_division_id("Cell Biology"))


In [ ]:
bookings.cur.execute("""SELECT rowid,name FROM booking_types WHERE name=='toto'""")
rows = bookings.cur.fetchall()
#rows[0][0]
#for row in rows:
#    print(row)
len(rows)

In [ ]:
try :
    bookings.cur.execute('''INSERT INTO booking_types (name) VALUES (?)''', ('service',))
except :
    pass



In [ ]:
bookings.create_booking_types(booking_types)


In [ ]:
del bookings